## Compiling functions with Cython

In [1]:
import pyximport

In [2]:
%load_ext Cython

In [3]:
def my_op(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [4]:
%%cython

def my_cop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [5]:
my_op

<function __main__.my_op>

In [6]:
my_cop

<function _cython_magic_ecbada9d02cb39603b728258c2e088b9.my_cop>

In [7]:
%timeit my_op([1,2,3,4,5])

100000 loops, best of 3: 3.03 µs per loop


In [8]:
%timeit my_cop([1,2,3,4,5])

1000000 loops, best of 3: 1.68 µs per loop


In [9]:
import numpy

In [10]:
numpy.array([1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [11]:
d = numpy.array([1,2,3,4,5])
( d ** 2 + 3 ) / 4.5

array([ 0.88888889,  1.55555556,  2.66666667,  4.22222222,  6.22222222])

In [12]:
%timeit ( d ** 2 + 3 ) / 4.5

The slowest run took 8.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.25 µs per loop


In [13]:
bigdata = numpy.random.randn(1000000)
bigdata.dtype

dtype('float64')

In [14]:
%timeit my_op(bigdata.copy())

1 loop, best of 3: 668 ms per loop


In [15]:
%timeit ( bigdata.copy() ** 2 + 3 ) / 4.5

100 loops, best of 3: 8.97 ms per loop


In [16]:
%timeit my_cop(bigdata.copy())

1 loop, best of 3: 493 ms per loop


In [17]:
%%cython
cimport numpy as np

def my_cop(np.ndarray[np.float_t, ndim=1] data):
    cdef int i

    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [18]:
%timeit my_cop(bigdata.copy())

100 loops, best of 3: 6.48 ms per loop


## Just-in-time optimization with numba

In [19]:
from numba import jit

In [20]:
@jit
def my_nbop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [21]:
%timeit my_nbop(bigdata.copy())

The slowest run took 83.64 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 3.78 ms per loop
